HTTP/2的目的是通过支持完整的请求与响应服用来减少延迟，通过有效压缩`HTTP`标头字段将协议开销降至最低，同时增加对请求优先级和服务器推送的支持。

为达成这些目标，HTTP/2还给我们带来了大量其他协议层面的辅助实现，例如新的流控制、错误处理和升级机制。

HTTP/2没有改动HTTP的应用语义。HTTP方法、状态代码、URI和标头字段等核心一如往常。不过，`HTTP/2`修改了数据格式化（分帧）以及在客户端与服务器间传输的方式。这两点统帅全局，通过新的分帧层向我们的应用隐藏了所有复杂性。因此，**所有现有的应用都可以不必修改而在新协议下运行**。

## HTTP/1.x

1. 客户端需要使用多个连接才能实现并发和缩短延迟
2. 不会压缩请求和响应标头，从而导致不必要的网络流量
3. 不支持有效的资源优先级，致使底层TCP连接的利用率低下

HTTP/2通过支持标头字段压缩和在**同一连接上进行多个并发交换**，让应用更有效地利用网络资源，减少感知的延迟时间。具体来说，它可以在对同一个连接上的请求和响应消息进行交错发送并为`HTTP`标头字段使用有效编码。HTTP/2还允许为请求设置优先级，让更重要的请求更快速地完成，从而进一步提升性能。出台地协议对网络更加友好，因为与`HTTP/1.x`相比，可以使用更少地TCP连接。

![](https://developers.google.com/web/fundamentals/performance/http2/images/binary_framing_layer01.svg?hl=zh-cn)

HTTP/2所有性能增强地核心在于新的二进制分帧层，它定义了如何封装HTTP消息并在客户端与服务器之间传输

这里所谓的“层”，指的是位于套接字接口与应用可见的高级HTTP API之间一个晋国优化的新编码机制，不同的是传输期间对它们的编码方式变了。HTTP/1.x协议以换行符作为纯文本的分隔符好，而HTTP/2将所有传输的信息分割为更小的消息和帧，并采用二进制格式对它们编码。

这样以来，客户端和服务器为了相互理解，都必须使用新的二进制编码机制：`HTTP/1.x`客户端无法理解只支持`HTTP/2`的服务器，反之亦然。不过不要紧，现有的应用不必担心这些变化，因为客户端和服务器会替我们完成必要的分帧工作。

新的二进制分帧机制改变了客户端与服务器之间交换数据的方式。为了说明这个过程，我们需要了解`HTTP/2`的三个概念：

数据流： 已建立的连接内的双向字节流，可以承载一条或多条消息。
消息： 与逻辑请求或响应消息对应的完整的一系列帧
帧： `HTTP/2`通信的最小单位，每个帧都包含帧头，至少也会标识处当前帧所属的数据流。

这些概念的关系总结如下：

1. **所有通信都在一个TCP连接上完成**，此链接可以承载任意数量的双向数据流
2. 每个数据流都有一个唯一的标识符和可选的优先级信息，用于承载双向消息
3. 每条消息都是一条逻辑`HTTP`消息（例如请求或响应），包含一个或多个帧
4. 帧是最小的通信单位，承载着特定类型的数据，例如HTTP标头、消息负载，等等。来自不同数据流的帧可以交错发送，然后再更具每个帧头的数据流标识符重新组装

![](https://developers.google.com/web/fundamentals/performance/http2/images/streams_messages_frames01.svg?hl=zh-cn)

简而言之，`HTTP/2`将HTTP协议通信分解为二进制编码帧的交换，这些帧对应着特定数据流中的消息，所有这些都在一个TCP连接内服用。这是`HTTP/2`协议所有其他功能和性能优化的基础

在`HTTP/1.x`中，如果客户端想要发起多个并行请求以提升性能，则必须使用多个TCP连接。这是`HTTP/1.x`交付模型的直接结果，该模型可以保证每个连接只交付一个响应（响应排队）。更糟糕的是，这种模型也会导致队首堵塞，从而造成底层TCP连接的效率低下。

`HTTP/2`新的二进制分帧层突破了这些限制，实现了完整的请求和响应服用：客户端和服务器可以将HTTP消息分解为互不依赖的帧，然后交错发送，最后再在另一端把它们重新组装起来。

![](https://developers.google.com/web/fundamentals/performance/http2/images/multiplexing01.svg?hl=zh-cn)

HTTP/2中新的二进制分帧层突破了这些限制，实现了完整的请求和响应服用：客户端和服务器可以将HTTP消息分解为互补依赖的帧，然后交错发送，最后再再另一端把它们重新组装起来。

将HTTP消息分解为独立的帧，交错发送，然后在另一端重新组装是HTTP2最重要的一项增强。实施上，这个机制会在整个网络技术栈中引发一系列连锁反应，从而带来巨大的性能提升，让我们可以：

1. 并行交错地发送多个请求，请求之间互不影响
2. 并行交错地发送多个响应，响应之间互补干扰
3. 使用一个连接并行地发送多个请求和响应
4. 消除不必要地延迟和提高现有地网络容量地利用率，从而减少页面加载时间

HTTP/2中新地二进制分帧层解决了`HTTP/1.x`中存在地队首堵塞问题，也消除了并i选哪个处理和发送请求及响应时对多个连接地依赖。

将HTTP消息分解为很多独立地帧之后，我们就可以复用多个数据流中的帧，客户端和服务器交错发送和传输这些帧的顺序就称为关键的性能决定因素。为了做到这一点，HTTP/2标准每个数据流都有一个关联的权重和依赖关系

1. 可以向每个数据流分配一个介于1至256之间的整数
2. 每个数据流与其他数据流之间可以存在显示依赖关系

数据流依赖关系和权重的组合让客户端可以构建和传递“优先级树“，表明它倾向于如何接受响应。反过来，服务器可以使用此信息通过控制`CPU`、内存和其他资源的分配设定数据流处理的优先级，在资源数据可用之后，带宽分配可以确保将高优先级响应以最优方式传输至客户端。

![](https://developers.google.com/web/fundamentals/performance/http2/images/stream_prioritization01.svg?hl=zh-cn)

HTTP/2 内的数据流依赖关系通过将另一个数据流的唯一标识符作为父项引用进行声明；如果忽略标识符，相应数据流将依赖于“根数据流”。声明数据流依赖关系指出，应尽可能先向父数据流分配资源，然后再向其依赖项分配资源。换句话说，“请先处理和传输响应 D，然后再处理和传输响应 C”。

共享相同父项的数据流（即，同级数据流）应按其权重比例分配资源。 例如，如果数据流 A 的权重为 12，其同级数据流 B 的权重为 4，那么要确定每个数据流应接收的资源比例，请执行以下操作：

1. 将所有权重求和：4 + 12 = 16
2. 将每个数据流权重除以总权重：A = 12/16, B = 4/16因此，数据流 A 应获得四分之三的可用资源，数据流 B 应获得四分之一的可用资源；数据流 B 获得的资源是数据流 A 所获资源的三分之一。我们来看一下上图中的其他几个动手示例：顺序为从左到右：
3. 数据流 A 和数据流 B 都没有指定父依赖项，依赖于显式“根数据流”；A 的权重为 12，B 的权重为 4。因此，根据比例权重：数据流 B 获得的资源是 A 所获资源的三分之一。
4. 数据流 D 依赖于根数据流；C 依赖于 D。因此，D 应先于 C 获得完整资源分配。权重不重要，因为 C 的依赖关系拥有更高的优先级。
5. 数据流 D 应先于 C 获得完整资源分配；C 应先于 A 和 B 获得完整资源分配；数据流 B 获得的资源是 A 所获资源的三分之一。
6. 数据流 D 应先于 E 和 C 获得完整资源分配；E 和 C 应先于 A 和 B 获得相同的资源分配；A 和 B 应基于其权重获得比例分配。

如上面的示例所示，数据流依赖关系和权重的组合明确表达了资源优先级，这是一种用于提升浏览性能的关键功能，网络中拥有多种资源类型，它们的依赖关系和权重各不相同。不仅如此，HTTP/2 协议还允许客户端随时更新这些优先级，进一步优化了浏览器性能。换句话说，我们可以根据用户互动和其他信号更改依赖关系和重新分配权重。

有了新的分帧机制后，HTTP/2 不再依赖多个 TCP 连接去并行复用数据流；每个数据流都拆分成很多帧，而这些帧可以交错，还可以分别设定优先级。因此，所有 HTTP/2 连接都是永久的，而且仅需要每个来源一个连接，随之带来诸多性能优势。

大多数 HTTP 传输都是短暂且急促的，而 TCP 则针对长时间的批量数据传输进行了优化。 通过重用相同的连接，HTTP/2 既可以更有效地利用每个 TCP 连接，也可以显著降低整体协议开销。不仅如此，使用更少的连接还可以减少占用的内存和处理空间，也可以缩短完整连接路径（即，客户端、可信中介和源服务器之间的路径）这降低了整体运行成本并提高了网络利用率和容量。 因此，迁移到 HTTP/2 不仅可以减少网络延迟，还有助于提高通量和降低运行成本。